In [1]:
from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

# Import the main FlowsheetBlock from IDAES. The flowsheet block will contain the unit model
from idaes.core import FlowsheetBlock

import idaes.logger as idaeslog

from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core.util.model_statistics import degrees_of_freedom

# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

# Import unit models from the model library
from idaes.generic_models.unit_models.distillation import TrayColumn
from idaes.generic_models.unit_models.distillation.condenser \
    import CondenserType, TemperatureSpec

In [2]:
from HFCS_bmimPF6_PR import configuration

In [3]:
# Create the ConcreteModel and the FlowsheetBlock, and attach the flowsheet block to it.
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False}) 

# Add properties parameter blocks to the flowsheet with specifications
m.fs.props = GenericParameterBlock(default=configuration)

2021-04-23 10:56:10 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [4]:
m.fs.unit = TrayColumn(default={
                       "number_of_trays": 5,
                       "feed_tray_location":2,
                       "condenser_type": CondenserType.totalCondenser,
                       "condenser_temperature_spec":
                           TemperatureSpec.atBubblePoint,
                       "property_package": m.fs.props,
                       "has_heat_transfer": False,
                       "has_pressure_change": False})

In [5]:
m.fs.props.PR_kappa['R32', 'R125'].fix(-0.002535)
m.fs.props.PR_kappa['R125', 'R32'].fix(-0.002535)
m.fs.props.PR_kappa['R32', 'bmimPF6'].fix(-0.01435)
m.fs.props.PR_kappa['bmimPF6', 'R32'].fix(0.9651)
m.fs.props.PR_kappa['bmimPF6', 'R125'].fix(0.6312)
m.fs.props.PR_kappa['R125', 'bmimPF6'].fix(0.0504)

# m.fs.props.PR_kappa['R32', 'bmimPF6'].fix(-0.05102)
# m.fs.props.PR_kappa['bmimPF6', 'R32'].fix(0.2839)
# m.fs.props.PR_kappa['bmimPF6', 'R125'].fix(1.6929)
# m.fs.props.PR_kappa['R125', 'bmimPF6'].fix(0.04738)

In [6]:
# Fix Heater H101 inlet conditions
m.fs.unit.feed.mole_frac_comp[0, "R32"].fix(0.20)
m.fs.unit.feed.mole_frac_comp[0, "R125"].fix(0.10)
m.fs.unit.feed.mole_frac_comp[0, "bmimPF6"].fix(0.7)

m.fs.unit.feed.flow_mol.fix(100)
m.fs.unit.feed.temperature.fix(298.15)
m.fs.unit.feed.pressure.fix(100000)

In [7]:
m.fs.unit.condenser.reflux_ratio.fix(1.1)
# m.fs.unit.condenser.reflux_ratio.fix(1.1)
m.fs.unit.condenser.condenser_pressure.fix(100000)

m.fs.unit.reboiler.boilup_ratio.fix(0.35)
# m.fs.unit.reboiler.control_volume.properties_out[0].temperature.fix(350)
# m.fs.unit.condenser.temperature.fix(286)
# m.fs.unit.condenser.vapor_outlet.temperature.fix(286)

In [8]:
print(degrees_of_freedom(m.fs.unit))
print(degrees_of_freedom(m.fs.unit.condenser))
print(degrees_of_freedom(m.fs.unit.reboiler))

0
6
6


In [ ]:
m.fs.unit.initialize() 

2021-04-23 10:56:14 [INFO] idaes.init.fs.unit: Begin initialization.
2021-04-23 10:56:14 [WARNING] idaes.init.fs.unit: Solver not provided. Default solver(ipopt) being used for initialization.
2021-04-23 10:56:14 [INFO] idaes.init.fs.unit.feed_tray: Begin initialization.
2021-04-23 10:56:14 [WARNING] idaes.init.fs.unit.feed_tray: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-23 10:56:14 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Starting initialization
2021-04-23 10:56:14 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-23 10:56:14 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Equilibrium temperature initialization completed.
2021-04-23 10:56:15 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-23 10:56:16 [INFO] idaes.init.fs.unit.feed_tray.properties_in_feed: 

2021-04-23 10:56:48 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-23 10:56:48 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Equilibrium temperature initialization completed.
2021-04-23 10:56:48 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-04-23 10:56:49 [INFO] idaes.init.fs.unit.rectification_section[1].properties_in_vap: Property initialization: optimal - Optimal Solution Found.
2021-04-23 10:56:49 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Starting initialization
2021-04-23 10:56:50 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-04-23 10:56:50 [INFO] idaes.init.fs.unit.rectification_section[1].properties_out: Equilibrium temperature initializati

2021-04-23 10:57:25 [INFO] idaes.init.fs.unit.stripping_section[4]: Mass and energy balance solve infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-23 10:57:28 [INFO] idaes.init.fs.unit.stripping_section[4]: Mass, energy and pressure balance solve infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-23 10:57:28 [INFO] idaes.init.fs.unit.stripping_section[4]: Initialization complete, status infeasible - Converged to a locally infeasible point. Problem may be infeasible..
2021-04-23 10:57:28 [INFO] idaes.init.fs.unit.stripping_section[5]: Begin initialization.
2021-04-23 10:57:28 [WARNING] idaes.init.fs.unit.stripping_section[5]: Solver not provided. Default solver(ipopt)  being used for initialization.
2021-04-23 10:57:28 [INFO] idaes.init.fs.unit.stripping_section[5].properties_in_liq: Starting initialization
2021-04-23 10:57:29 [INFO] idaes.init.fs.unit.stripping_section[5].properties_in_liq: Dew and bubble p

In [ ]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [ ]:
results = solver.solve(m)


status = solver.solve(m, tee = True)

In [ ]:
m.display()